In [1]:
import pandas as pd

In [5]:
from os import listdir
from os.path import join

listdir('data/appraisal/filled')

['Nurit.xlsx', 'Tzipy.xlsx', 'Anat.xlsx', 'Dan.xlsx', 'Nehoray.xlsx']

In [139]:
apps = pd.concat([pd.read_excel(join('data/appraisal/filled', f), usecols=(0,1,2), names=('uri_unsh_no_clean','appraisal','comments')) for f in listdir('data/appraisal/filled')]) 

In [140]:
apps['appraisal'] = apps['appraisal'].astype(bool)

In [141]:
apps.appraisal.value_counts() / apps.shape[0]

True     0.763647
False    0.236353
Name: appraisal, dtype: float64

In [142]:
apps.comments.value_counts()

internal 404                  112
homepage, no carbon dating     89
Later date                     32
irrelevant                     25
server error                   24
no content                     24
generic site                   17
Earlier date                   12
Redirects to a dead link       11
later date                     11
early date                      9
earlier date                    4
Homepage, no carbon dating      4
wall garden                     3
*                               2
Wall garden                     2
walled garden                   1
404                             1
Server Error                    1
irrelrvant                      1
redirects to a dead link        1
Name: comments, dtype: int64

In [145]:
apps['comments'] = apps.comments.str.lower()
apps.loc[apps.comments=='wall garden', 'comments'] = 'walled garden'
apps.loc[apps.comments=='early date', 'comments'] = 'earlier date'
apps.loc[apps.comments=='irrelrvant', 'comments'] = 'irrelevant'
apps.loc[(((~apps.appraisal)) & (apps.comments.isna()) | (apps.comments=='')) , 'comments'] = 'irrelevant'


In [147]:
apps.comments.value_counts()

internal 404                  112
homepage, no carbon dating     93
irrelevant                     56
later date                     43
earlier date                   25
server error                   25
no content                     24
generic site                   17
redirects to a dead link       12
walled garden                   6
*                               2
Name: comments, dtype: int64

In [148]:
lut = pd.read_pickle('lang_uri_time_wb_avail_all.pkl.gz', compression='gzip')
macd_all = pd.read_pickle('data/macd_all.pkl.gz', compression='gzip')
lut['uri_unsh_no_clean'] = lut.uri_unshrtn_raw.combine_first(lut.uri_unshrtn_bitly.combine_first(lut.uri_unshrtn.combine_first(lut.uri)))
lut['uri_mem_avail'] = lut.uri.map(~macd_all.drop_duplicates(subset='uri').set_index('uri').first_snap.isna()).fillna(False)
lut['uri_unsh_no_clean_mem_avail'] = lut.uri_unsh_no_clean.map(~macd_all.drop_duplicates(subset='uri').set_index('uri').first_snap.isna()).fillna(False)
curl = pd.read_csv('data/curl_results_parsed.csv.gz', compression='gzip')
lut['status_code'] = lut.uri_unsh_no_clean.map(curl.drop_duplicates(subset='url').set_index('url').status_code)
lut['content_type'] = lut.uri_unsh_no_clean.map(curl.drop_duplicates(subset='url').set_index('url').content_type)
lut.head().T

,0,1,2,3,4
lang,zh,zh,zh,zh,zh
source,wikipedia,wikipedia,wikipedia,wikipedia,wikipedia
timestamp,2018-08-22 13:21:14,2018-06-25 15:15:42,2017-11-10 21:53:43,2017-11-10 21:53:43,2017-11-10 21:53:43
uri,https://web.archive.org/web/20150626164255/htt...,https://web.archive.org/web/20140727005451/htt...,https://archive.is/20140828023420/http://www.w...,https://web.archive.org/web/20140803200140/htt...,https://web.archive.org/web/20140808044248/htt...
uri_cln,web.archive.org/web/20150626164255/http://www....,web.archive.org/web/20140727005451/http://trav...,archive.is/20140828023420/http://www.worldjour...,web.archive.org/web/20140803200140/http://www....,web.archive.org/web/20140808044248/http://m.al...
tld,archive.org,archive.org,archive.is,archive.org,archive.org
is_shortened,False,False,False,False,False
uri_unshrtn,NaN,NaN,NaN,NaN,NaN
uri_cln_unshrtn,web.archive.org/web/20150626164255/http://www....,web.archive.org/web/20140727005451/http://trav...,archive.is/20140828023420/http://www.worldjour...,web.archive.org/web/20140803200140/http://www....,web.archive.org/web/20140808044248/http://m.al...
uri_unshrtn_bitly,NaN,NaN,NaN,NaN,NaN


In [149]:
lap = lut.merge(apps, how='inner')
lap.uri_unsh_no_clean.value_counts()

http://www.independent.co.uk/news/world/middle-east/israelgaza-conflict-the-myth-of-hamass-human-shield-9619810.html                                                                                                                                                                                                                                                                                                                                                                                                  737
https://www.amnesty.org/en/latest/news/2014/07/israelgaza-conflict-questions-and-answers/                                                                                                                                                                                                                                                                                                                                                                                                                     

In [150]:
(lap.groupby(['source', 'appraisal']).uri_unsh_no_clean.nunique()
 .unstack().astype(int)
 .assign(ratio = lambda x: np.round((x[True] /(x[False]+x[True])), 2))
)

appraisal,False,True,ratio
source,,,
google,134,478,0.78
twitter,226,1038,0.82
wikipedia,62,431,0.87
youtube,42,44,0.51


In [151]:
(lap.groupby(['is_shortened', 'appraisal']).uri_unsh_no_clean.nunique()
 .unstack().astype(int)
 .assign(ratio = lambda x: np.round((x[True] /(x[False]+x[True])), 2))
)

appraisal,False,True,ratio
is_shortened,,,
False,353,1126,0.76
True,65,548,0.89


In [152]:
lap['is_unshrtn'] = lap.uri_cln_unshrtn!=lap.uri_cln
(lap.groupby(['is_unshrtn', 'appraisal']).uri_unsh_no_clean.nunique()
 .unstack().astype(int)
 .assign(ratio = lambda x: np.round((x[True] /(x[False]+x[True])), 2))
)

appraisal,False,True,ratio
is_unshrtn,,,
False,363,1126,0.76
True,55,548,0.91


In [170]:
(lap.groupby(['source', 'is_unshrtn', 'appraisal']).uri_unsh_no_clean.nunique()
 .unstack().fillna(0).astype(int)
 .assign(ratio = lambda x: np.round((x[True] /(x[False]+x[True])), 2))
)

appraisal             False  True  ratio
source    is_unshrtn                    
google    False         134   478   0.78
          True            0     0    NaN
twitter   False         175   523   0.75
          True           53   546   0.91
wikipedia False          62   431   0.87
youtube   False          40    42   0.51
          True            2     2   0.50

In [153]:
import numpy as np

In [154]:
lut['archived'] = lut[['uri_cln_wb_avail', 'uri_cln_unshrtn_wb_avail', 'uri_mem_avail', 'uri_unsh_no_clean_mem_avail']].any(axis=1)


In [155]:
def get_values(g):
    return ','.join(list(g.unique()))
tlds_to_remove = ['fb.me', 'facebook.com', 'youtu.be', 'youtube.com']
uris_for_appr = lut[(lut.uri_unsh_no_clean.isin(apps.uri_unsh_no_clean)) & (~lut.tld_unshrtn.isin(tlds_to_remove))].groupby(['uri_unsh_no_clean', 'tld_unshrtn', 'status_code']).agg({'archived': np.max, 'source': get_values}).reset_index()

In [156]:
uris_for_appr = uris_for_appr.merge(apps)

In [157]:
uris_for_appr.head()

,uri_unsh_no_clean,tld_unshrtn,status_code,archived,source,appraisal,comments
0,feedproxy.google.com/~r/DefenseAndDiplomacy/~3...,google.com,301.0,False,twitter,False,server error
1,http://20to30.com/video/israeli-soldier-victim/,20to30.com,200.0,True,twitter,False,"homepage, no carbon dating"
2,http://24sata.info/vijesti/vijesti/svijet/2036...,24sata.info,200.0,False,google,True,NaN
3,http://7online.com/217058/,7online.com,301.0,False,twitter,False,"homepage, no carbon dating"
4,http://85fashionkitty.blogspot.co.il,blogspot.co.il,302.0,True,youtube,False,"homepage, no carbon dating"


In [158]:
uris_for_appr.to_csv('data/appraisal/appraisal_join.csv', index=False)

In [159]:
(uris_for_appr
 .groupby(['source', 'appraisal']).size().unstack().fillna(0)
 .astype(int)
 .assign(ratio = lambda x: np.round((x[True] /(x[False]+x[True])), 2))
)

appraisal,False,True,ratio
source,,,
google,122,129,0.51
twitter,174,397,0.70
"twitter,google",10,325,0.97
wikipedia,30,105,0.78
"wikipedia,google",2,12,0.86
"wikipedia,twitter",27,288,0.91
"wikipedia,twitter,google",0,11,1.00
"wikipedia,youtube",1,9,0.90
"wikipedia,youtube,twitter",2,6,0.75


In [160]:
shared = ((uris_for_appr.source.str.contains(',')) & (uris_for_appr.appraisal)).sum() / uris_for_appr.source.str.contains(',').sum()
not_shared = ((~uris_for_appr.source.str.contains(',')) & (uris_for_appr.appraisal)).sum() / (~uris_for_appr.source.str.contains(',')).sum()
shared, not_shared

(0.9233983286908078, 0.648)

In [161]:
(uris_for_appr
 .groupby(['status_code', 'appraisal']).size().unstack().fillna(0)
 .astype(int)
 .assign(ratio = lambda x: np.round((x[True] /(x[False]+x[True])), 2))
)

appraisal,False,True,ratio
status_code,,,
200.0,154,810,0.84
301.0,211,451,0.68
302.0,42,50,0.54


In [162]:
(uris_for_appr
 .groupby(['source', 'comments']).size().unstack().fillna(0).astype(int)
)

comments,*,earlier date,generic site,"homepage, no carbon dating",internal 404,irrelevant,later date,no content,redirects to a dead link,server error,walled garden
source,,,,,,,,,,,
google,0,13,1,20,23,33,32,0,2,0,0
twitter,0,2,8,50,53,11,2,18,7,20,5
"twitter,google",1,1,0,3,2,2,2,1,0,0,0
wikipedia,0,5,0,5,11,2,3,1,1,3,0
"wikipedia,google",0,0,0,0,0,0,2,0,0,0,0
"wikipedia,twitter",1,3,7,3,9,1,1,3,0,0,0
"wikipedia,youtube",0,0,0,0,1,0,0,0,0,0,0
"wikipedia,youtube,twitter",0,0,0,0,2,0,0,0,0,0,0
youtube,0,0,1,10,3,7,0,0,2,2,1
